# Spark 101

# Exercises
Using the repo setup directions, setup a new local and remote repository named spark-exercises. The local version of your repo should live inside of ~/codeup-data-science. This repo should be named spark-exercises

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

# Imports

In [1]:
# standard imports
import pandas as pd
import numpy as np

#import spark for python! 
import pyspark
from pyspark.sql.functions import concat, lit
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
from pyspark.sql.functions import month, year, quarter, round
from pyspark.sql.functions import asc, desc
from pydataset import data
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import min, max, sum, count, mean, avg
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

#create the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/05 09:50:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Create a spark data frame that contains your favorite programming languages.

* The name of the column should be language
* View the schema of the dataframe
* Output the shape of the dataframe
* Show the first 5 records in the dataframe


In [2]:
np.random.seed(452)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), language=np.random.choice(("python", 'java', 'javascript', 'r', 'c++', 'sql'), 20))
)

pandas_dataframe

,n,language
0,0,javascript
1,1,python
2,2,r
3,3,c++
4,4,python
5,5,python
6,6,r
7,7,python
8,8,javascript
9,9,python


In [3]:
# create spark dataframe 
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[n: bigint, language: string]

In [4]:
# printSchema
df.schema

StructType([StructField('n', LongType(), True), StructField('language', StringType(), True)])

In [5]:
# number of rows
df.count()

20

In [6]:
# number of columns
len(df.columns)

2

In [7]:
# Show the first 5 records in the dataframe
df.show()

+---+----------+
|  n|  language|
+---+----------+
|  0|javascript|
|  1|    python|
|  2|         r|
|  3|       c++|
|  4|    python|
|  5|    python|
|  6|         r|
|  7|    python|
|  8|javascript|
|  9|    python|
| 10|       c++|
| 11|      java|
| 12|    python|
| 13|         r|
| 14|         r|
| 15|       c++|
| 16|       c++|
| 17|    python|
| 18|       c++|
| 19|      java|
+---+----------+



# 2. Load the mpg dataset as a spark dataframe.

a. Create 1 column of output that contains a message like the one below:


- The 1999 audi a4 has a 4 cylinder engine.
For each vehicle.

b. Transform the trans column so that it only contains either manual or auto.

In [8]:
df = spark.createDataFrame(data('mpg'))
df

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

In [9]:
df.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

In [10]:
df.select('class').show()

+-------+
|  class|
+-------+
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|compact|
|midsize|
|midsize|
|midsize|
|    suv|
|    suv|
+-------+
only showing top 20 rows



- The 1999 audi a4 has a 4 cylinder engine.

In [11]:
df.select(df.manufacturer, df.model, df.displ, df.year, df.cyl, df.trans, df.drv, df.cty, df.hwy, df.fl, df['class'],
          concat(lit('The '),df.year, lit(' '),
           df.manufacturer, lit(' '),
           df.model, lit(' has a '),
           df.cyl, lit(' cylinder engine.')).alias('vehicle_info')
).show(truncate=False) # truncate will expand the column

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+
|manufacturer|model             |displ|year|cyl|trans     |drv|cty|hwy|fl |class  |vehicle_info                                                  |
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------------------------------------------------+
|audi        |a4                |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.                     |
|audi        |a4                |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008 audi a4 has a 4 cylinder e

In [12]:
df = df.select(df.manufacturer, df.model, df.displ, df.year, df.cyl, df.trans, df.drv, df.cty, df.hwy, df.fl, df['class'],
    regexp_extract('trans', r'(\D+)\(', 1).alias('auto_manual'),
)

In [13]:
df.filter(df['auto_manual'] == 'manual').show()

+------------+-------------------+-----+----+---+----------+---+---+---+---+-------+-----------+
|manufacturer|              model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|auto_manual|
+------------+-------------------+-----+----+---+----------+---+---+---+---+-------+-----------+
|        audi|                 a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|     manual|
|        audi|                 a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|     manual|
|        audi|                 a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|     manual|
|        audi|         a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|     manual|
|        audi|         a4 quattro|  2.0|2008|  4|manual(m6)|  4| 20| 28|  p|compact|     manual|
|        audi|         a4 quattro|  2.8|1999|  6|manual(m5)|  4| 17| 25|  p|compact|     manual|
|        audi|         a4 quattro|  3.1|2008|  6|manual(m6)|  4| 15| 25|  p|compact|     manual|
|   chevrolet|           corve

## 3. Load the tips dataset as a spark dataframe.

In [14]:
tips_df = spark.createDataFrame(data('tips'))
tips_df

DataFrame[total_bill: double, tip: double, sex: string, smoker: string, day: string, time: string, size: bigint]

In [15]:
tips_df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [16]:
tips_df.sort(tips_df.smoker.desc()).show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     20.29|3.21|  Male|   Yes|Sat|Dinner|   2|
|     34.65|3.68|  Male|   Yes|Sun|Dinner|   4|
|     11.35| 2.5|Female|   Yes|Fri|Dinner|   2|
|     23.33|5.65|  Male|   Yes|Sun|Dinner|   2|
|     17.51| 3.0|Female|   Yes|Sun|Dinner|   2|
|     45.35| 3.5|  Male|   Yes|Sun|Dinner|   3|
|     15.38| 3.0|Female|   Yes|Fri|Dinner|   2|
|     23.17| 6.5|  Male|   Yes|Sun|Dinner|   4|
|     38.01| 3.0|  Male|   Yes|Sat|Dinner|   4|
|     40.55| 3.0|  Male|   Yes|Sun|Dinner|   2|
|      44.3| 2.5|Female|   Yes|Sat|Dinner|   3|
|      20.9| 3.5|Female|   Yes|Sun|Dinner|   3|
|     10.59|1.61|Female|   Yes|Sat|Dinner|   2|
|     30.46| 2.0|  Male|   Yes|Sun|Dinner|   5|
|     22.42|3.48|Female|   Yes|Sat|Dinner|   2|
|     18.15| 3.5|Female|   Yes|Sun|Dinner|   3|
|     12.16| 2.2|  Male|   Yes|Fri| Lunch|   2|
|      23.1| 4.0|  Male|   Yes|Sun|Dinne

### a. What percentage of observations are smokers?

In [17]:
tips_df.filter(tips_df.smoker == 'Yes').show()

+----------+----+------+------+----+------+----+
|total_bill| tip|   sex|smoker| day|  time|size|
+----------+----+------+------+----+------+----+
|     38.01| 3.0|  Male|   Yes| Sat|Dinner|   4|
|     11.24|1.76|  Male|   Yes| Sat|Dinner|   2|
|     20.29|3.21|  Male|   Yes| Sat|Dinner|   2|
|     13.81| 2.0|  Male|   Yes| Sat|Dinner|   2|
|     11.02|1.98|  Male|   Yes| Sat|Dinner|   2|
|     18.29|3.76|  Male|   Yes| Sat|Dinner|   4|
|      3.07| 1.0|Female|   Yes| Sat|Dinner|   1|
|     15.01|2.09|  Male|   Yes| Sat|Dinner|   2|
|     26.86|3.14|Female|   Yes| Sat|Dinner|   2|
|     25.28| 5.0|Female|   Yes| Sat|Dinner|   2|
|     17.92|3.08|  Male|   Yes| Sat|Dinner|   2|
|     19.44| 3.0|  Male|   Yes|Thur| Lunch|   2|
|     32.68| 5.0|  Male|   Yes|Thur| Lunch|   2|
|     28.97| 3.0|  Male|   Yes| Fri|Dinner|   2|
|      5.75| 1.0|Female|   Yes| Fri|Dinner|   2|
|     16.32| 4.3|Female|   Yes| Fri|Dinner|   2|
|     40.17|4.73|  Male|   Yes| Fri|Dinner|   4|
|     27.28| 4.0|  M

In [18]:
#calculate the average smokers
tips_df.filter(tips_df.smoker == 'Yes').select(
    avg(tips_df.smoker)
).show()

+-----------+
|avg(smoker)|
+-----------+
|       null|
+-----------+



In [19]:
#groupby/groupBy
tips_df.groupby(tips_df.smoker).agg(count(tips_df.smoker)).show()  #.agg(mean(tips_df['smoker'])).show()

+------+-------------+
|smoker|count(smoker)|
+------+-------------+
|    No|          151|
|   Yes|           93|
+------+-------------+



In [20]:
tips_df.select((count(when(col('smoker') == 'Yes', True))/count('*')*100).alias('smoker_yes')).show()

+------------------+
|        smoker_yes|
+------------------+
|38.114754098360656|
+------------------+



In [21]:
#groupby/groupBy
smoker_percentage = tips_df.select((count(when(col('smoker') == 'Yes', True)
                                         ) / count('*')).alias('smoker_percentage')
                                  ).collect()[0]['smoker_percentage'] * 100
print(f"The percentage of smokers is: {smoker_percentage}%")

The percentage of smokers is: 38.114754098360656%


### b. Create a column that contains the tip percentage

In [22]:
#use min, max, and calculate average highway mileage
tips_df.select(
    min(tips_df.tip),
    max(tips_df.tip),
    avg(tips_df.tip),
    mean(tips_df.tip),
    count(col('tip'))
).show(5)

+--------+--------+----------------+----------------+----------+
|min(tip)|max(tip)|        avg(tip)|        avg(tip)|count(tip)|
+--------+--------+----------------+----------------+----------+
|     1.0|    10.0|2.99827868852459|2.99827868852459|       244|
+--------+--------+----------------+----------------+----------+



Explain that a tip is calculated by multiplying the total amount of the bill by the tip percentage expressed as a decimal. ° For instance: The total bill for a service is $27. The percentage to tip is typically 15% (0.15), 18% (0.18), or 20% (0.20).

In [23]:
tips_df.select(
    expr('((tip/total_bill)*100)')
).show()

+--------------------------+
|((tip / total_bill) * 100)|
+--------------------------+
|        5.9446733372572105|
|        16.054158607350097|
|        16.658733936220845|
|         13.97804054054054|
|        14.680764538430255|
|         18.62396204033215|
|         22.80501710376283|
|        11.607142857142858|
|        13.031914893617023|
|        21.853856562922868|
|         16.65043816942551|
|        14.180374361883155|
|        10.181582360570687|
|        16.277807921866522|
|        20.364126770060686|
|        18.164967562557923|
|         16.16650532429816|
|        22.774708410067525|
|        20.624631703005306|
|        16.222760290556902|
+--------------------------+
only showing top 20 rows



In [24]:
tips_df = tips_df.withColumn(
    'tip_percentage', # new column name
    round((col('tip')/col('total_bill')*100)) # how the new column is created
)

In [25]:
tips_df.show()

+----------+----+------+------+---+------+----+--------------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_percentage|
+----------+----+------+------+---+------+----+--------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|           6.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|          16.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|          17.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|          14.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|          15.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|          19.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|          23.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|          12.0|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|          13.0|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|          22.0|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|          17.0|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|          14.0|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|        

### c. Calculate the average tip percentage for each combination of sex and smoker.

In [26]:
tips_df.groupBy(tips_df.sex).agg(mean(tips_df.smoker)).show()

+------+-----------+
|   sex|avg(smoker)|
+------+-----------+
|Female|       null|
|  Male|       null|
+------+-----------+



In [27]:
#groupby and pivot
tips_df.groupby('sex').pivot('smoker').avg('tip_percentage').show()

+------+------------------+------------------+
|   sex|                No|               Yes|
+------+------------------+------------------+
|Female|15.685185185185185|18.242424242424242|
|  Male|16.103092783505154|15.283333333333333|
+------+------------------+------------------+



In [28]:
#groupby and pivot
tips_df.groupby('sex').pivot('smoker').mean('tip_percentage').show()

+------+------------------+------------------+
|   sex|                No|               Yes|
+------+------------------+------------------+
|Female|15.685185185185185|18.242424242424242|
|  Male|16.103092783505154|15.283333333333333|
+------+------------------+------------------+



## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [29]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



### * Convert the temperatures to fahrenheit.

In [30]:
weather.withColumn('temp_max', round(col('temp_max')*(9/5)+32)).show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    55.0|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    51.0|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    53.0|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    54.0|     5.6| 4.7|   rain|
|2012-01-05|          1.3|    48.0|     2.8| 6.1|   rain|
|2012-01-06|          2.5|    40.0|     2.2| 2.2|   rain|
|2012-01-07|          0.0|    45.0|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    50.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|    49.0|     5.0| 3.4|   rain|
|2012-01-10|          1.0|    43.0|     0.6| 3.4|   rain|
|2012-01-11|          0.0|    43.0|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|    43.0|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|    41.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|    40.0|     0.6| 5.3|   snow|
|2012-01-15|  

In [31]:
tips_df = tips_df.withColumn(
    'tip_percentage', # new column name
    round((col('tip')/col('total_bill')*100)) # how the new column is created
)

In [32]:
weather.withColumn('temp_max', round(col('temp_max')*(9/5)+32)
                  ).withColumn('temp_min', round(col('temp_min')*(9/5)+32)).show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    55.0|    41.0| 4.7|drizzle|
|2012-01-02|         10.9|    51.0|    37.0| 4.5|   rain|
|2012-01-03|          0.8|    53.0|    45.0| 2.3|   rain|
|2012-01-04|         20.3|    54.0|    42.0| 4.7|   rain|
|2012-01-05|          1.3|    48.0|    37.0| 6.1|   rain|
|2012-01-06|          2.5|    40.0|    36.0| 2.2|   rain|
|2012-01-07|          0.0|    45.0|    37.0| 2.3|   rain|
|2012-01-08|          0.0|    50.0|    37.0| 2.0|    sun|
|2012-01-09|          4.3|    49.0|    41.0| 3.4|   rain|
|2012-01-10|          1.0|    43.0|    33.0| 3.4|   rain|
|2012-01-11|          0.0|    43.0|    30.0| 5.1|    sun|
|2012-01-12|          0.0|    43.0|    29.0| 1.9|    sun|
|2012-01-13|          0.0|    41.0|    27.0| 1.3|    sun|
|2012-01-14|          4.1|    40.0|    33.0| 5.3|   snow|
|2012-01-15|  

### * Which month has the most rain, on average?

In [58]:
(
    weather.withColumn("month", month("date"))
    .groupBy("month")
    .agg(mean("precipitation").alias("most_rain_on_avg"))
    .sort(desc("most_rain_on_avg"))
    .first()
)

Row(month=11, most_rain_on_avg=5.354166666666667)

### * Which year was the windiest?

In [34]:
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(mean("wind").alias("windiest"))
    .sort(desc("windiest"))
    .first()
)

Row(year=2012, windiest=3.4008196721311483)

### * What is the most frequent type of weather in January?

In [35]:
weather.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|     4.4|     0.6| 5.3|   snow|
|2012-01-15|  

In [36]:
(
    weather.withColumn("month", month("date"))
    .filter(month("date") == 1)
    .groupBy("month")
    .agg(count(col("weather")).alias("most_freq_type_of_weather_jan"))
    .show()
)

+-----+-----------------------------+
|month|most_freq_type_of_weather_jan|
+-----+-----------------------------+
|    1|                          124|
+-----+-----------------------------+



In [37]:
(
    weather.withColumn("month", month("date"))
    .filter(month("date") == 1)
    .groupBy("weather")
    .count()
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|drizzle|   10|
|   rain|   35|
|    sun|   33|
|   snow|    8|
|    fog|   38|
+-------+-----+



In [38]:
weather.withColumn("month", month("date"))\
    .filter(month("date") == 1)\
    .groupBy("weather")\
    .count()\
    .orderBy('count', ascending=False)\
    .show(1)


+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
+-------+-----+
only showing top 1 row



### * What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [39]:
(
    weather.filter(month("date") == 7)
    .filter(year("date") == 2013)
    .where(col('weather') == 'sun')
    .groupBy("weather")
    .agg(min("temp_min"),max('temp_max'))
    .show()
)


+-------+-------------+-------------+
|weather|min(temp_min)|max(temp_max)|
+-------+-------------+-------------+
|    sun|         11.1|         31.7|
+-------+-------------+-------------+



In [40]:
#groupby and pivot
(
    weather.filter(year("date") == 2013)
    .filter(month("date") == 7)
    .agg(min('temp_min'))
    .show()
)

+-------------+
|min(temp_min)|
+-------------+
|         11.1|
+-------------+



In [41]:
(
    weather.filter(year("date") == 2013 & 2014)
    .filter(month("date") ==7)
    .where(col('weather') == 'sun')
    .groupBy('weather', year('date'))
    .agg(min('temp_min'), max('temp_max'))
    .show()
)

+-------+----------+-------------+-------------+
|weather|year(date)|min(temp_min)|max(temp_max)|
+-------+----------+-------------+-------------+
|    sun|      2012|          9.4|         26.7|
+-------+----------+-------------+-------------+



In [42]:
(
    weather.filter(year("date").isin([2013, 2014]))
    .filter(month("date") == 7)
    .where(col('weather') == 'sun')
    .groupBy('weather', year('date'))
    .agg(avg('temp_min').alias('avg_min_temp'), avg('temp_max').alias('avg_max_temp'))
    .show()
)

+-------+----------+------------------+------------------+
|weather|year(date)|      avg_min_temp|      avg_max_temp|
+-------+----------+------------------+------------------+
|    sun|      2013|13.981481481481483|26.585185185185193|
|    sun|      2014|14.400000000000002|            27.092|
+-------+----------+------------------+------------------+



### * What percentage of days were rainy in q3 of 2015?

In [43]:
total_days = weather.filter(year("date") == 2015)\
                    .filter(quarter("date") == 3)\
                    .count()

rainy_days = weather.filter(year("date") == 2015)\
                     .filter(quarter("date") == 3)\
                     .filter(col('weather') == 'rain')\
                     .count()

percentage = (rainy_days / total_days) * 100

print("Percentage of rainy days in Q3 2015: ", percentage)


Percentage of rainy days in Q3 2015:  2.1739130434782608


### * For each year, find what percentage of days it rained (had non-zero precipitation).

In [44]:
total_days_2012 = weather.filter(year("date") == 2012)\
                    .count()

rainy_days_2012 = weather.filter(year("date") == 2012)\
                    .filter(col('weather') == 'rain')\
                    .count()

percentage = (rainy_days / total_days) * 100

print("Percentage of rainy days in 2012: ", percentage)

Percentage of rainy days in 2012:  2.1739130434782608


In [45]:
total_days

92

In [46]:
rainy_days

2

In [47]:
total_day = weather.withColumn("year", year("date"))\
                    .groupBy("year")\
                    .count()

rainy_day = weather.withColumn('year', year('date'))\
                    .filter(col('weather') == 'rain')\
                    .groupBy('year')\
                    .count().alias('count1')

# percentage = (rainy_days.count / total_days.count) * 100


In [48]:
total_day = total_day.withColumnRenamed('count', 'cnt1')

In [49]:
rainy_day = rainy_day.withColumnRenamed('count', 'cnt2')

In [50]:
new_weather = total_day.join(rainy_day, 'year', 'inner')

In [51]:
new_weather.withColumn('percentage_of_rainy_days', ((col('cnt2'))/(col('cnt1'))*100)).show()

+----+----+----+------------------------+
|year|cnt1|cnt2|percentage_of_rainy_days|
+----+----+----+------------------------+
|2012| 366| 191|       52.18579234972678|
|2013| 365|  60|       16.43835616438356|
|2014| 365|   3|       0.821917808219178|
|2015| 365|   5|        1.36986301369863|
+----+----+----+------------------------+



In [52]:
from pyspark.sql.functions import year, when, col

In [53]:
# Calculate total days and rainy days for each year
weather_with_year = weather.withColumn("Year", year("date"))
total_days_per_year = weather_with_year.groupBy("Year").count()
total_days_per_year = total_days_per_year.withColumnRenamed("count", "Total_Days")
rainy_days_per_year = weather_with_year.where(col('precipitation') > 0).groupBy("Year").count()
rainy_days_per_year = rainy_days_per_year.withColumnRenamed("count", "Rainy_Days")

# Join the two dataframes on Year
result = total_days_per_year.join(rainy_days_per_year, ["Year"], "inner")

# Calculate the percentage of rainy days
result = result.withColumn("Percentage_of_Rainy_Days", (col("Rainy_Days") / col("Total_Days")) * 100)

result.show()


+----+----------+----------+------------------------+
|Year|Total_Days|Rainy_Days|Percentage_of_Rainy_Days|
+----+----------+----------+------------------------+
|2012|       366|       177|       48.36065573770492|
|2013|       365|       152|       41.64383561643836|
|2014|       365|       150|        41.0958904109589|
|2015|       365|       144|       39.45205479452055|
+----+----------+----------+------------------------+



In [54]:
weather_with_year.show()

+----------+-------------+--------+--------+----+-------+----+
|      date|precipitation|temp_max|temp_min|wind|weather|Year|
+----------+-------------+--------+--------+----+-------+----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|2012|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|2012|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|2012|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|2012|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|2012|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|2012|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|2012|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|2012|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|2012|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|2012|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|2012|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|2012|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    su

In [55]:
total_days_per_year.show()

+----+----------+
|Year|Total_Days|
+----+----------+
|2012|       366|
|2013|       365|
|2014|       365|
|2015|       365|
+----+----------+



In [56]:
rainy_days_per_year.show()

+----+----------+
|Year|Rainy_Days|
+----+----------+
|2012|       177|
|2013|       152|
|2014|       150|
|2015|       144|
+----+----------+



In [57]:
result.show()

+----+----------+----------+------------------------+
|Year|Total_Days|Rainy_Days|Percentage_of_Rainy_Days|
+----+----------+----------+------------------------+
|2012|       366|       177|       48.36065573770492|
|2013|       365|       152|       41.64383561643836|
|2014|       365|       150|        41.0958904109589|
|2015|       365|       144|       39.45205479452055|
+----+----------+----------+------------------------+

